<center>
     <font size="6">RSA</font>
</center>



* Rivest, R.; A. Shamir; L. Adleman **(1978)**. *A Method for Obtaining Digital Signatures and Public-Key Cryptosystems* . Communications of the ACM 21 (2): 120–126

* The Original RSA Patent as filed with the U.S. Patent Office by Rivest; Ronald L. (Belmont, MA), Shamir; Adi (Cambridge, MA), Adleman; Leonard M. (Arlington, MA), **December 14, 1977**, U.S. Patent 4,405,829.

---


+ <a href="#RSA_1" id="vRSA_1">Criptosistema RSA</a>
    * <a href="#RSA_1.1" id="vRSA_1.1">Puesta en funcionamiento</a>
    * <a href="#RSA_1.2" id="vRSA_1.2">Cifrado y descifrado de mensajes</a>
+ <a href="#RSA_2" id="vRSA_2">Seguridad</a>
    * <a href="#RSA_2.1" id="vRSA_2.1">Factorización</a>
        - <a href="#RSA_2.1.1" id="vRSA_2.1.1">Factorización de Fermat</a>
        - <a href="#RSA_2.1.2" id="vRSA_2.1.2">Método p − 1 </a>
        - <a href="#RSA_2.1.3" id="vRSA_2.1.3">Criba cuadrática</a>
        - <a href="#RSA_2.1.4" id="vRSA_2.1.4">Otros métodos de factorización</a>
        - [RSA Factoring Challenge](https://en.wikipedia.org/wiki/RSA_Factoring_Challenge)
        - Integer factorization calculator (https://www.alpertron.com.ar/ECM.HTM)
    * <a href="#RSA_2.2" id="vRSA_2.2">Algunos ataques al uso del RSA</a>
         - <a href="#RSA_2.2.1" id="vRSA_2.2.1">Exponente compartido</a>
         - <a href="#RSA_2.2.2" id="vRSA_2.2.2">Módulo compartido</a>
         - <a href="#RSA_2.2.3" id="vRSA_2.2.3">Timing / Power attacks</a>
         - <a href="#RSA_2.2.4" id="vRSA_2.2.4">Criptograma escogido (reto-respuesta)</a>
         - <a href="#RSA_2.2.5" id="vRSA_2.2.5">Insuficiente control del formato</a>
         - <a href="#RSA_2.2.6" id="vRSA_2.2.6">Error provocado</a>
         - [Recovering cryptographic keys from partial information, by example](https://eprint.iacr.org/2020/1506)
         - <a href="#RSA_2.2.7" id="vRSA_2.2.7">Primos no aleatorios</a>
         - <a href="#RSA_2.2.8" id="vRSA_2.2.8">Cajas negras</a>
+ <a href="#RSA_3" id="vRSA_3">Recomendaciones</a>

---

---

# <a href="#vRSA_1" id="RSA_1">Criptosistema RSA</a>

Criptosistema RSA: $({\cal M},{\cal C},{\cal K},{\cal E},{\cal D})$

* ${\cal M}=\mathbb Z_n$.
* ${\cal C}=\mathbb Z_n$.
* ${\cal K}=\{ (n,p,q,e,d)\,:\, n=pq, \ p,q$ primos impares diferentes, $ed\equiv 1 \mod \varphi(n)\}$.
* Para $k=(n,p,q,e,d)\in{\cal K}$, $x,y\in\mathbb Z_n$:
  - $e_k(x)=x^e\mod n$,
  - $d_k(y)=y^d\mod n$.



[RFC3447 Public-Key Cryptography Standards (PKCS) \#1: RSA Cryptography](https://datatracker.ietf.org/doc/html/rfc3447)



---
## <a href="#vRSA_1.1" id="RSA_1.1">Puesta en funcionamiento</a>


Cada usuario realiza los siguientes pasos:


* Elige $e$. Generalmente se toma $e$ primo e
  * igual a  65537 ($2^{16}+1$) si su uso es para cifrar
  * igual a  3 ($2^{1}+1$) si su uso es para firmar
* Genera dos números primos aleatorios diferentes, $p$ y $q$ tales que $\gcd(e,p-1)=1$, $\gcd(e,q-1)=1$.
* Calcula $n=pq$.
* Calcula $d=e^{-1}\mod MCM(p-1,q-1)$. (Algoritmo extendido de Euclides)
* Calcula 
    * $d_p\equiv d \mod (p-1)$,
    * $d_q\equiv d \mod (q-1)$,
    * $p_{inv}\equiv p^{-1}\mod q$
    * $q_{inv}\equiv q^{-1}\mod p$

**Clave pública**: $\{e,n\}$. 
**Clave privada**: $\{d,p,q,d_p,d_q,q_{inv}\}$.


In [ ]:
e = 2^16+1

NdeBits = 1024*(4) # bits del módulo

r1 = randint(0,2^(NdeBits/2))
r2 = randint(0,2^(NdeBits/2))
p = next_probable_prime(r1)
while gcd(e,p-1) != 1:
    print( ".")
    p = next_probable_prime(p+2)

q = next_probable_prime(r2)
while gcd(e,q-1) != 1:
    print( "+")
    q = next_probable_prime(q+2)

n = p*q
phi = (p-1)*(q-1)
d = mod(e^-1,phi)
dp = mod(d,p-1)
dq = mod(d,q-1)
q_inv = mod(q^-1,p)

print("n =", n)
print("e =", e)
print()
print("p =", p)
print("q =", q)
print("d =", d)
print("dp =", dp)
print("dq =", dq)
print("q_inv =", q_inv)

## <a href="#vRSA_1.2" id="RSA_1.2">Cifrado y descifrado de mensajes</a>

### Cifrado / verificación de firma (Primitiva)

Sea $m$ una cadena de bits interpretada como un entero $0\leq m < n$: 

$$c=m^e\mod n$$

### Descifrado / firma (Primitiva usando TCR)

Para recuperar $m$ se calcula:
    $$M_p=c^{d_p}\mod p, \qquad M_q=c^{d_q}\mod q$$
    y  se resuelve el sistema de congruencias:
    $$
    \left. \begin{array}{rl}
        &m\equiv M_p \mod p \\
        &m\equiv M_q \mod q
      \end{array}\right\}
    $$
    cuya solución es
    $$
    m=M_p q_{inv}q+M_q p_{inv}p \mod n.
    $$
    
Resolver el sistema de congruencias es equivalente a calcular:
    $$
    h= (M_p-M_q)q_{inv} \mod p
    $$
    y 
    $$
    m=M_q+q\,h \mod n
    $$

In [ ]:
from time import time


m=randint(0, 2^NdeBits);

m=mod(m,n); #Necesario para que trabaje en Zn, si no tarda mucho

c=mod(m^e,n);
cp_aux=mod(c,p); #Necesario para que trabaje en Zp, si no tarda mucho
cq_aux=mod(c,q); #Necesario para que trabaje en Zq, si no tarda mucho

numero_de_iteraciones=100
print('Número de iteraciones:', numero_de_iteraciones)
inicio=time();
for i in range(numero_de_iteraciones):
    c=mod((m+i)^e,n);
    m_rsa=mod(c^d,n);
    if m+i!=m_rsa:
        print ("Error RSA")
print ("Tiempo RSA c^d mod n:",time()-inicio);

inicio=time()
for i in range(numero_de_iteraciones):
    c=mod((m+i)^e,n);
    cp_aux=mod(c,p); #Necesario para que trabaje en Zp, si no tarda mucho
    cq_aux=mod(c,q); #Necesario para que trabaje en Zq, si no tarda mucho
    mp=Integer(mod(cp_aux^dp,p));
    mq=Integer(mod(cq_aux^dq,q));
    h=mod((mp-mq)*Integer(q_inv),n);
    m_tcr=mod((mq+q*h),n);
    if m+i!=m_tcr:
        print( "Error TCR")
    hp=mod((mp-mq)*Integer(q_inv),p);
    mp_tcr=mod((mq+q*Integer(hp)),n);
    if m+i!=mp_tcr:
        print( "Error Mp TCR")

print( "Tiempo RSA CRT:      ",time()-inicio);

#print "m=",m;
#print "c=",c;
#print "m_rsa=",m_rsa;
#print "mp=",mp;
#print "mq=",mq;
#print "h=",h;
#print "m_tcr=",m_tcr;
#print "hp=",hp;
#print "mp_tcr=",mp_tcr;

### PKCS #1: RSA Cryptography Specifications Version 2.2
https://www.rfc-editor.org/rfc/rfc8017.html




#### RSA-OAEP Optimal asymmetric encryption padding
![RSA-OAEP Optimal asymmetric encryption padding](RSA-oaep1200.png)
* El padding consiste en una cadena de 0 o más bytes 0x00 acabada necesariamente con el byte 0x01.
* lHash es el hash de los parámetros que se le pasan, si no hay parámetros es el hash de una cadena vacía.
* mask generation function MGF($Z$,$l$) devuelve una cadena de $l$ bytes generados de la siguiente forma:
    * Sea $T$ una cadena vacía
    * Para $i=0,...,\frac{l}{hLen-1}$  
    
        $T=T\|Hash(Z\|i)$ $\quad$ ($i$ entero de 32 bits)
        
    * Salida los primeros $l$ bytes de $T$.         
* La función $Hash(\cdot)$ que devuelve $hLen$ bytes se ha de acordar previamente al igual que el tamaño de la semilla. 
            
* $m$ se considera un elemento de  $\mathbb Z_n$ y se cifra usando la primitiva criptográfica.



Para descifrar un mensaje cifrado recibido se siguen los pasos:

* Se descifra $c$ utilizando la primitiva criptográfica (se recomienda la versión TCR) 
    y la clave privada del receptor del mensaje.
    
    
* Se comprueba que $m$ tiene el formato adecuado (el padding y lHash recuperados son los correctos). Si es así, se extrae $M$.


---

#### RSA-PSS Probabilistic signature scheme
![RSA-PSS Probabilistic signature scheme](RSA-PSS-1200.png)

* $Padding_1$ consiste en una cadena de 8 bytes 0x00.
* $Padding_2$ consiste en una cadena de 0 o más bytes 0x00 acabada necesariamente con el byte 0x01.
* La función $Hash(\cdot)$ se ha de acordar previamente al igual que el tamaño de $Salt$. 
* $m$ se considera un elemento de  $\mathbb Z_n$ y se ***descifra*** usando la primitiva criptográfica (se recomienda la versión TCR), el resultado es la firma $f$.



Para verificar la firma de un mensaje:

* Se ***cifra*** $f$ utilizando la primitiva criptográfica y la clave pública del firmante del mensaje.
* Se comprueba que $m$ tiene el formato adecuado (el byte final coincide con 0xBC). 

Si es así:
* se extrae $Salt$,
* con el documento firmado se calcula $mHash$,
* se calcula el hash $H'$ de $Padding_1 \| mHash \| Salt$.
* Si $H' = H$ entonces la firma es válida.


---
# <a href="#vRSA_2" id="RSA_2">Seguridad</a>

## <a href="#vRSA_2.1" id="RSA_2.1">Factorización</a>


### <a href="#vRSA_2.1.1" id="RSA_2.1.1">Factorización de Fermat</a>

Sea $n$ un  entero positivo impar tal que se puede escribir
como producto de dos enteros positivos, $n=ab$. Entonces 
$$
n=ab=s^2-t^2, \hskip1cm s=\frac{a+b}{2}, \hskip.3cm t=\frac{a-b}{2}.
$$

Por otra parte si $n$ es un entero positivo impar tal que se 
puede escribir como $n=s^2-t^2$, entonces
$$
n=s^2-t^2=ab, \hskip1cm a=s+t, \hskip.3cm b=s-t.
$$

Para que este método de factorización sea útil $t$ ha de ser pequeño.



[**Algorithm for factoring some RSA and Rabin moduli**](http://arxiv.org/abs/1303.5226) Omar Khadir

Si $\vert p-q\vert\leq 2^{\frac{k+5}{4}}$, $k$ tamaño del módulo en bits, entonces, si $n_0=\lfloor\sqrt{n}\rfloor+1$, $I=n_0^2-n$ es un cuadrado prefecto y $p=n_0-\sqrt{I}$, $q=n_0+\sqrt{I}$.

In [ ]:
NdeBits = 1024*2
r1 = randint(0, 2^NdeBits)
k = randint(0, 2^(NdeBits/2))
p1 = next_probable_prime(r1)
p2 = next_probable_prime(r1+k)
n = p1*p2

n0 = floor(sqrt(n))+1
I = n0^2-n
q1 = n0-sqrt(I)
q2 = n0+sqrt(I)

print(p1)
print(p2)
print("\n-------------------------- diferencia entre p y q:\n ", p2-p1)
print("\nPrimos calculados:")
print(q1)
print(q2)
print(q1-p1)
print(q2-p2)

https://crt.sh/?id=7581884753

> Non-authoritative answer:
>
> Name:	www.whitehatsecurity.be
>
> Address: 95.211.203.221


> Non-authoritative answer:
>
> www.accessdenied.be	canonical name = accessdenied.be.
>
> Name:	accessdenied.be
>
> Address: 95.211.203.221


In [ ]:
aux = '''00:ee:3e:c0:ea:b6:3d:53:ab:5f:18:8b:05:7c:6a:
                    99:15:87:c1:8c:69:e0:ee:16:73:b1:a9:4f:df:4c:
                    81:e7:cd:66:8e:8d:8d:47:3d:d6:27:3d:6e:f4:ab:
                    cc:38:69:26:5f:8a:7c:4f:0f:13:8c:33:6f:ab:52:
                    40:be:88:fe:dd:cc:76:ba:e4:3b:da:84:0a:f7:c5:
                    4a:40:99:53:fb:89:28:71:f4:9a:b2:94:ca:56:1b:
                    3b:54:6d:14:06:d2:8e:36:10:0b:1a:50:1b:32:f5:
                    98:b0:5e:0c:58:c8:99:2d:aa:84:9a:da:ca:14:84:
                    7a:1d:a5:85:83:bb:d2:a0:95:19:97:22:f8:77:25:
                    07:5b:35:c5:20:09:8b:87:5d:63:01:07:d5:ce:70:
                    50:78:5c:eb:ea:04:cc:a8:38:ea:22:10:f4:4d:42:
                    6a:04:b7:38:c3:f8:75:e1:23:a1:16:ea:4e:50:96:
                    3a:a2:87:83:7a:bb:3b:76:64:e9:8a:f3:99:cc:f5:
                    dd:a0:89:70:80:03:b1:55:62:ad:37:0b:bd:1c:da:
                    93:25:53:85:23:d2:c6:52:c1:dc:1f:0d:83:51:16:
                    50:b5:4b:b2:b3:bd:03:f9:82:87:4d:fa:30:9d:d7:
                    12:84:d0:df:9d:22:83:b8:a7:b0:9f:5e:63:44:ea:
                    71:31'''
for _aux in [' ', '\n', ':']:
    aux = "".join(aux.split(_aux))
    # print(aux)

n = int(aux,16)
n0 = floor(sqrt(n))+1
I = n0^2-n
q1 = n0-sqrt(I)
q2 = n0+sqrt(I)

print("\nPrimos calculados:")
print(q1)
print(q2)
print(q2-q1)
next_probable_prime(q1)

### <a href="#vRSA_2.1.2" id="RSA_2.1.2">Método  $p-1$</a>

Sea $p$ primo un factor de $n$ tal que todas las potencias de primo divisoras de $p-1$ son menores que un cierto $B$.

Entonces $(p-1)\vert B!$

Calculemos ahora $a\equiv 2^{B!} \mod n$. Como $p\vert n$ tenemos que $a\equiv 2^{B!} \mod p$.

Por el teorema de Fermat $1\equiv 2^{p-1} \mod p$ y como $(p-1)\vert B!$ entonces $a\equiv 1 \mod p$ y por lo tanto $p\vert (a-1)$. Teniendo en cuenta que $p\vert n$ podemos concluir que $p$ divide al máximo común divisor de $a-1$ y $n$, $p\vert d=\gcd(a-1,n)$.

En consecuencia $d$ es un factor no trivial de $n$. Hemos reducido el problema de factorizar $n$ al problema de factorizar $d$ y $\frac{n}{d}$.

La complejidad de este algoritmo es ${\cal O}(B\log B\log^2 n +\log^3 n)$. Si $B\simeq \sqrt n$ su complejidad es la misma que si nos dedicáramos a dividir por todos los posibles divisores.

Para que este método de factorización sea útil $p-1$ ha de tener solamente factores pequeños. 

Una forma de evitar este ataque es elegir los primos del RSA  de la forma
$p=2p_1+1$, $q=2q_1+1$, $p_1$ y $q_1$ primos.



In [ ]:
NdeBits=32;
r1=randint(0,2^NdeBits);
r2=randint(0,2^NdeBits);
p = next_probable_prime(r1)
q = next_probable_prime(r2)
n = p*q

print("p-1=", factor(p-1))
print("q-1=", factor(q-1))

a = 2
x = 1
B = 2^20
print("Tomo B = 2^20 =", B)

while x < B and gcd(a-1,n)==1:
    x = x+1
    a = mod(a^x, n)


if x<B:
    print("Factorizado!")
else:
    print("Error!")

print(n)
print('a=', a)
print('mcd(a-1,n)=', gcd(a-1, n))
print( p)
print( q)


### <a href="#vRSA_2.1.3" id="RSA_2.1.3">Criba cuadrática</a>

Dados $n=pq$, $y$ y $z$ tales que $y\not\equiv \pm z\mod n$ y 
$y^2\equiv z^2 \mod n$ entonces $(y-z,n)=p\ \textrm{ó}\ q$.

Se generan $x_1,\ldots,x_m$. Para cada número se calcula 
$x_i^2\mod n$ que se intenta factorizar sobre una base de primos 
$\{2,3,\ldots,p_k\}$. En el caso de ser posible tendremos
$$
x_i^2\mod n=\Pi_{j=1}^k p_j^{e_j}.
$$

Cuando obtengamos $k+1$, por eliminación gausiana podremos escribir
uno de los vectores $(e_1,\ldots,e_k)$ como combinación lineal 
(módulo 2) de los otros. Entonces 
$$
\Pi_{i=1}^m \left(x_i^{2}\right)^{b_i} \equiv 
\Pi_{j=1}^k p_j^{c_j}\mod n,
$$ 
siendo $b_i$ 0 ó 1 (dependiendo
de si pertenece o no a la combinación lineal) y $c_j$ la suma de
las multiplicidades de los primos.


Definiendo 
$$
y\equiv\Pi_{i=1}^m x_i^{b_i} \mod n
\hskip1cm \textrm{y} \hskip1cm 
z\equiv  \Pi_{j=1}^k p_j^{c_j/2}\mod n
$$ entonces 
$$
y^2\equiv z^2 \mod n.
$$


Con $m=100\,000\,000$ y $k=200\,000$ la criba se puede realizar 
en unos pocos segundos (5-10) obteniendo poquísimos números 
cuyo cuadrado sea factorizable.

La complejidad del algoritmo es 
${\cal O}\left(\mbox{e}^{\left(1+\circ(1)\right)
\sqrt{\log n\log\log n}}\right)$.


In [ ]:
p = 3571
q = 4409
n = p*q
print("n=", n)

Base = list(primes(1,12))
print('Base de primos:', Base)
print()
NumeroDePrimos = len(Base)
NumeroDeFactorizacionesABuscar = NumeroDePrimos+4
NumeroDeFactorizaciones = 0
NumeroDeIntentos = 0

exponenteMatrix = [[0 for i in range(0,NumeroDePrimos+2)] for i in range(0, NumeroDeFactorizacionesABuscar)]
exponenteMatrixBinaria = [[0 for i in range(0,NumeroDePrimos)] for i in range(0, NumeroDeFactorizacionesABuscar)]

while NumeroDeFactorizaciones<NumeroDeFactorizacionesABuscar:
    NumeroDeIntentos = NumeroDeIntentos+1
    exponente = list([0 for i in  range(0,NumeroDePrimos)])
    x = randint(1, n)
    x2 = mod(x^2, n)
    for i in range(NumeroDePrimos):
        p = Base[i]
        while mod(x2,p) == 0:
            exponente[i] = exponente[i]+1
            x2 = x2/p
    if x2 == 1:
        print( "Numero de Intentos: ", NumeroDeIntentos, "; X=", x, "; X^2 mod n =", mod(x^2,n))
        for i in range(len(Base)):
            p = Base[i]
            exponenteMatrix[NumeroDeFactorizaciones][i] = exponente[i]
            exponenteMatrixBinaria[NumeroDeFactorizaciones][i] = exponente[i]%2
            #print p, "^", exponente[i]
        exponenteMatrix[NumeroDeFactorizaciones][NumeroDePrimos] = x
        exponenteMatrix[NumeroDeFactorizaciones][NumeroDePrimos+1] = Integer(mod(x^2, n))
        NumeroDeFactorizaciones = NumeroDeFactorizaciones+1

print( "\n Numero de Intentos: ", NumeroDeIntentos)
for i in range(NumeroDeFactorizacionesABuscar):
    print( i,  exponenteMatrixBinaria[i], exponenteMatrix[i], "   ", factor(exponenteMatrix[i][NumeroDePrimos+1]))

In [ ]:
solucion = [0,2]
X = 1
exponenteSolucion = Matrix([0 for i in range(0,NumeroDePrimos)])
for i in solucion:
    X = X*exponenteMatrix[i][len(Base)]
    exponenteSolucion = exponenteSolucion+Matrix(exponenteMatrix[i][:-2])
Y = 1
for i in range(len(Base)):
    Y = Y*(Base[i]**(exponenteSolucion[0][i]/2))
gcd(n, X-Y)

### <a href="#vRSA_2.1.4" id="RSA_2.1.4">Otros métodos de factorización</a>

* **Curvas elípticas** ${\cal O}\left(\mbox{e}^{\left(1+\circ(1)\right)
    \sqrt{2\log p\log\log p}}\right)$ siendo $p$ el menor divisor 
    primo de $n$.
* **Criba del cuerpo de números**
    ${\cal O}\left(\mbox{e}^{\left(1.92+\circ(1)\right)
    (\log n)^{1/2} (\log\log n)^{2/3}}\right)$. Es el que mejor se comporta 
    cuando $n$ crece.



---
## <a href="#vRSA_2.2" id="RSA_2.2">Ataques al uso del RSA</a>



### <a href="#vRSA_2.2.1" id="RSA_2.2.1">Exponente compartido</a>

Supongamos que enviamos el mismo mensaje $m$ a tres personas cuyo
exponente público es $e=3$:
$$
c_1\equiv m^3 \mod n_1,
$$
$$
c_2\equiv m^3 \mod n_2,
$$
$$
c_3\equiv m^3 \mod n_3.
$$

Si $\gcd(n_1,n_2)=\gcd(n_1,n_3)=\gcd(n_2,n_3)=1$ por el teorema chino 
existe una única solución del sistema
$$
x\equiv c_1 \mod n_1,
$$
$$
x\equiv c_2 \mod n_2,
$$
$$
x\equiv c_3 \mod n_3,
$$
módulo $n=n_1n_2n_3$. 

La solución es **exactamente igual a** (no solo
congruente con) $m^3$. Podemos hallar $m$ calculando la raíz
cúbica en los reales (es fácil y rápido) del resultado 
obtenido aplicando el teorema chino de los restos.


El coste para hallar $m$ a partir de los
criptogramas y las claves públicas es
${\cal O}\left(\log^3(n_1)+\log^2(n)\right)$ siendo $n=\prod_{i=1}^e
n_i$. 


Si los módulos no son relativamente primos, cosa altamente 
improbable, podemos factorizarlos (su $mcd$ nos da el factor 
primo que comparten.)

**Nota:**
	Al usar RSA-OAEP cada $m$ debería ser diferente, evitando este ataque. 



In [2]:
NdeBitsDelModulo=2048*2
limite=2^(NdeBitsDelModulo/2);
r1=randint(0,limite);
r2=randint(0,limite);
p1=next_probable_prime(r1);
p2=next_probable_prime(r2);
n1=p1*p2;
r1=randint(0,limite);
r2=randint(0,limite);
p1=next_probable_prime(r1);
p2=next_probable_prime(r2);
n2=p1*p2;
r1=randint(0,limite);
r2=randint(0,limite);
p1=next_probable_prime(r1);
p2=next_probable_prime(r2);
n3=p1*p2;


Z1= IntegerModRing(n1);
Z2= IntegerModRing(n2);
Z3= IntegerModRing(n3);


e=3;
m=mod(mod(mod(randint(0,limite^2),n1),n2),n3);
m1=Z1(m);
m2=Z2(m);
m3=Z3(m);

cc1=m1^e
cc2=m2^e
cc3=m3^e

print("\ncriptograma y módulo 1: ",cc1,'\n', n1);
print("\ncriptograma y módulo 2: ",cc2,'\n', n2);
print("\ncriptograma y módulo 3: ",cc3,'\n', n3);


c1=Integer(cc1);
c2=Integer(cc2);
c3=Integer(cc3);

#---------------------
#Empieza ataque
#---------------------
print("\n---------------- Empiezo los cálculos para descifrar------------------")
t0=cputime()
s=(crt(crt(c1,c2,n1,n2),c3,n1*n2,n3))^(1/e);
t1=cputime()

print("\nmensaje original=",m);
print("\nmensaje recuperado=",s);
print("\nDiferencia:",Integer(s)-Integer(m));

print("\n Tiempo=",t1-t0)


criptograma y módulo 1:  12178857428955156754883191044623919455113256555020247320407898934944463846451928680571941022074533976240645341867527328317244055171883527000646818429574133960816777492308083648155713565429928653290985365085558382046280837126451203568193442547806387197071405533683999085638563558082895982727945961269617516411818532956716400682305968276998846884480421775577276029144096451979014533432563727284911446189185762825787327268038013238633168096938841653882083163249888692156276021543294449378024970604455232871502181766201150639026873136981628846452398571968405736005202783959881908984916071050481024140904440507459928223650687995243298408787731518280669719822357908663733894763340630713500181844664488984142620034002698447113769739593680606093684500780981459583576816677724680215381625565339392540781896451984226088307383448919797338495499972347101634885657579569018465916311906565951803702963684622406450361203780329289447523198152094144127534780345412549607645878547899397292545

In [1]:
NdeBitsDelModulo=2048*2
limite=2^(NdeBitsDelModulo/2);
r1=randint(0,limite);
r2=randint(0,limite);
p1=next_probable_prime(r1);
p2=next_probable_prime(r2);
n1=p1*p2;
r1=randint(0,limite);
r2=randint(0,limite);
p1=next_probable_prime(r1);
p2=next_probable_prime(r2);
n2=p1*p2;
r1=randint(0,limite);
r2=randint(0,limite);
p1=next_probable_prime(r1);
p2=next_probable_prime(r2);
n3=p1*p2;


Z1= IntegerModRing(n1);
Z2= IntegerModRing(n2);
Z3= IntegerModRing(n3);


e=5;
m=mod(mod(mod(randint(0,limite^2),n1),n2),n3);
m1=Z1(m);
m2=Z2(m);
m3=Z3(m);

cc1=2381135004837897289190972124556231417118807930731324069160257956664435609464314385680469250470620831138095008900646;
cc2=2499873093349932324023281425895296431137480523032368295855507740581860865528598028799191990665092208197509522428214;
cc3=3632473745988792738802194052469335467505548049564875725126918423915403348043645271089535418777120141385580888901037;
cc4=1409220620617001057501573888283159677104961770966421376214595069656842417134099099261984297868300102501573653994925;
cc5=12270886775846611899442389328947778782582612652536625498026593376049816567444568574813082551421753150567283738309639;

n1 = 11704458880696591489703356592616416025127868098410953766007322413765303922671395383720842616703077269781776882240487;
n2 = 17234261282515109030920373387929821699941371306961933980268800658182839692020822375533773555017682851968911081539163;
n3 = 19053471012090780990243272591945771901765983282293268088579846661520394095256752218097009967231753236434269475701791;
n4 = 22610770261657789496058163182213778222550168328189289540160032215867782655240728693716233181590330679713873208985719;
n5 = 21950753793319115751513988943054963225104568326742396944258659289484943230064914465252162060935004886578442146573447;

print("\ncriptograma y módulo 1: ",cc1,'\n', n1);
print("\ncriptograma y módulo 2: ",cc2,'\n', n2);
print("\ncriptograma y módulo 3: ",cc3,'\n', n3);


c1=Integer(cc1);
c2=Integer(cc2);
c3=Integer(cc3);
c4=Integer(cc4);
c5=Integer(cc5);

#---------------------
#Empieza ataque
#---------------------
print("\n---------------- Empiezo los cálculos para descifrar------------------")
t0=cputime()
a=crt(c1, c2, n1, n2)
b=crt(a, c3, n1*n2, n3)
c=crt(b, c4, n1*n2*n3, n4)
d=crt(c, c5, n1*n2*n3*n4, n5)
s=d^(1/e);
t1=cputime()

print("\nmensaje original=",m);
print("\nmensaje recuperado=",s);
print("\nDiferencia:",Integer(s)-Integer(m));

print("\n Tiempo=",t1-t0)


criptograma y módulo 1:  2381135004837897289190972124556231417118807930731324069160257956664435609464314385680469250470620831138095008900646 
 11704458880696591489703356592616416025127868098410953766007322413765303922671395383720842616703077269781776882240487

criptograma y módulo 2:  2499873093349932324023281425895296431137480523032368295855507740581860865528598028799191990665092208197509522428214 
 17234261282515109030920373387929821699941371306961933980268800658182839692020822375533773555017682851968911081539163

criptograma y módulo 3:  3632473745988792738802194052469335467505548049564875725126918423915403348043645271089535418777120141385580888901037 
 19053471012090780990243272591945771901765983282293268088579846661520394095256752218097009967231753236434269475701791

---------------- Empiezo los cálculos para descifrar------------------

mensaje original= 842733298633209905772254829943934798773553007654179211662966366323367555547548450080179620126397285699772040588378499464758200

### <a href="#vRSA_2.2.2" id="RSA_2.2.2">Módulo compartido</a> </h3>

Una gran organización podría estar tentada de distribuir
entre sus miembros un par de exponentes, uno público y su
correspondiente exponente privado, y un módulo común a
todos los usuarios, pero no los valores de los primos que lo
componen. 

En esta situación podemos distinguir dos tipos de ataques,
según su procedencia:

* <em><span style="text-decoration: underline;">Atacante externo</span></em>
    Sean $c_1$ y $c_2$ los criptogramas resultantes de cifrar, con las claves públicas RSA $\{e_1, n\}$ y $\{e_2, n\}$  tales que $\gcd(e_1,e_2)=1$, un mismo mensaje $0\leq m<n$.

    Entonces, el tiempo necesario para hallar $m$ a partir de los criptogramas y las claves públicas es ${\cal O}\left(\log^3(n)\right).$
    
    Si $\gcd(e_1,e_2)=1$ podemos hallar $r$ y $s$ tales que $e_1 r+e_2 s=1$. Entonces
    $$
    c_1^rc_2^s\equiv m^{e_1 r}m^{e_2 s} \equiv m^{e_1 r+e_2 s}
    \equiv m \mod n.
    $$
    Las operaciones más costosas son las exponenciaciones modulares.


* <em><span style="text-decoration: underline;">Atacante interno</span></em>
    El conocimiento de un exponente público y su correspondiente exponente privado permite calcular los otros exponentes privados, incluso factorizar el módulo.

    Sean $n$ un módulo RSA, $3\leq e_{_A},d_{_A}<\varphi(n)$ enteros tales que $e_{_A} d_{_A}\equiv 1 \mod \varphi(n)$, y $3\leq e_{_B}<\varphi(n)$ un entero tal que $\gcd(e_{_B},\varphi(n))=1$.

    Busquemos un entero $\alpha$ con las siguientes propiedades:
    
    - $\alpha=k\varphi(n)$
    
    - $\gcd(\alpha, e_{_B})=1$.

    Entonces, por Bezout, podremos hallar $r$ y $s$ tales que $\alpha\,r+e_{_B}s=1$. Como $\alpha$ es un múltiplo de $\varphi(n)$, tomando módulo $\varphi(n)$ podremos concluir que $e_{_B}s\equiv 1\mod\varphi(n)$.  

    Definamos 
    $$
    g_0=e_{_A} d_{_A}-1, \hskip1cm h_0=\gcd(g_0,e_{_B}),
    $$ 
    y recursivamente
    $$
    g_i=\frac{g_{i-1}}{h_{i-1}}, \hskip1cm h_i=\gcd(g_i, e_{_B}).
    $$
    Notemos que $g_i$ siempre es entero ya que $h_{i-1}$ es un divisor de $g_{i-1}$. Sea $t$ el menor entero tal que $h_t=1$, (observemos que si $h_k\neq 1$ entonces $h_k\geq 2$ y $g_{k+1}\leq g_{k}/2$ y, por lo tanto, en un número de pasos del orden de $\log e_{_A} d_{_A}-1$ tendremos que alguna $h_j$    será 1.)

    Tal como hemos tomado $t$,  $h_t=1$ y, por lo tanto, $\gcd(g_t, e_{_B})=1$. Por otra parte $g_t=\dfrac{e_{_A} d_{_A}-1}{h_0\ldots h_{t-1}}= \dfrac{\lambda\varphi(n)}{h_0 \ldots h_{t-1}}$;     como $h_i=\gcd(g_i, e_{_B})$ y $\gcd(\varphi(n), e_{_B})=1$ tenemos que ningún $h_i$ divide a $\varphi(n)$ y, por lo tanto, que todos los $h_i$ dividen a $\lambda$. En consecuencia, $g_t$ es un múltiplo de $\varphi(n)$ (por la misma razón, no puede ser igual a 1).

    Resumiendo, $g_t$ cumple los requisitos pedidos para $\alpha$.

    El coste para hallar un entero $d_{_B}$ tal que $e_{_B} d_{_B}\equiv 1 \mod \varphi(n)$ es ${\cal O}\left(\log^3(n)\right)$.


In [3]:
NdeBitsDelModulo=2048;
limite=2^(NdeBitsDelModulo/2);
r1=randint(0,limite);
r2=randint(0,limite);
p=next_probable_prime(r1);
q=next_probable_prime(r2);
n=170538320697436872549809649066816068525278315503585613616926600658279661841794052814556310517678241518461467436380671665425021924276297381700142880232528452054970212543680296514864304929580458367386643270944790798417254655492298435638635963807216643697407017271104923429535053148910544192213065464757330795991;

e1=631;
e2=419;


m=mod(randint(0,limite),n);
c1=143648910760608541629655797863669233249654290192667946449353863958718782714466913671809733769863151633971910117649232741134345266577856389088427448917448410834321695399557954613004302086099726507346203118256286782283363963612698368541323343424249017272433470667797237669747178579607409233975301057631819285331;
c2=76253833939695580915043884230689522837459339989253913296900449103458516804925477653246549436825807710527383835373924232343152282510426950888503598042316911900489760021159070955189673766986093870581802468375295538753692677065391430507487638986710818627192198957468318822117728898517948974739108685564351309302;

#---------------------
#Empieza ataque
#---------------------
t0=cputime()

bezout=xgcd(e1,e2)
print(bezout)
m_rec=mod(mod(c1^bezout[1],n)*mod(c2^bezout[2],n),n)
t1=cputime()

print("mensaje original=",m);
print("mensaje recuperado=",m_rec);
print("\n Tiempo=",t1-t0)

(1, 168, -253)
mensaje original= 72724490251203265870779553369349142516051387771804133334253340403075563761624562620687831635283611646174991392479241832871740509487856162327692393562060734891694886656141772791983583143873664218634964402964721359555360068877259175997743885303300753684617454091053445298399003665161185206670281018900939585631
mensaje recuperado= 777777777777777777777777777777770000000000000000000000000000000001111111111111111111111111111111111111666666666666666666666666666666666666667777777777777777777777777777777777777777

 Tiempo= 0.0010419999999990992


In [ ]:
NdeBitsDelModulo=2048;
limite=2^(NdeBitsDelModulo/2);
r1=randint(0,limite);
r2=randint(0,limite);
p=next_probable_prime(r1);
q=next_probable_prime(r2);
n=p*q;
phi=(p-1)*(q-1);

e1=2^16+1;
d1=Integer(mod(e1^-1,phi));

e2=2*randint(0,limite)+1;
while gcd(e2,phi)!=1:
    e2=2*randint(0,limite)+1;
    
    
#---------------------
#Empieza ataque
#---------------------
t0=cputime()
g=e1*d1-1;
h=gcd(g,e2);
print("g, h: ", g,h);
while h!=1:
    g=g/h;
    h=gcd(g,e2);
    print("g, h: ", g,h);

bezout=xgcd(e2,g);

d2=bezout[1];



m=mod(randint(0,limite),n);
c2=mod(m^e2,n);
m_rec=mod(c2^d2,n)
t1=cputime()


print("\nmensaje original=",m)
print("\nmensaje recuperado=",m_rec)
print("\n Tiempo=",t1-t0)

### <a href="#vRSA_2.2.3" id="RSA_2.2.3">Timing / Power attacks</a>

Para calcular  $a^m \mod n$, $a$, $m>0$ y $n>0$ enteros, se considera a expressión del exponente en base 2, $m=b_r2^r+\dots+b_12+b_0$, donde $b_i\in\{ 0,1\}$  y $b_r=1$, y se usa el siguiente algoritmo: 

```python
y = 1
i = r
while i >= 0:
    y = (y**2)%n
    if b[i] == 1:
        y = (a*y)%n
    i = i-1
return y
```

Este algoritmo consume en cada iteración más tiempo/energía si el bit del exponente es 1 que si es 0. Mediante un análisis estadístico del tiempo/energía consumido es posible calcular
el exponente:

![Sin modificación](PowerAttacks1.png)
*Darrel Hankerson, Alfred Menezes, Scott Vanstone* 
Guide to Elliptic Curve Cryptography.
*Figure 5.13. Power trace for a sequence of addition (S) and double (D) operations on an elliptic curve over a prime field. Points were represented using Jacobian coordinates. The traces were obtained from an SC140 DSP processor core.*


Para evitar este ataque se puede utilizar el método de Montgomery
para la exponenciación (Montgomery Powering Ladder):

```python
R0 = R1 = 1
i = r
while i >= 0:
    if b[i] == 0:
        R1 = (R0*R1)%n
        R0 = (R0**2)%n
    else:
        R0 = (R0*R1)%n
        R1 = (R1**2)%n 
    i = i-1
return R0
```


Este algoritmo consume en cada iteración el mismo tiempo/energía 
independientemente del valor del bit del exponente:

![Con modificación](PowerAttacks2.png)
*Darrel Hankerson, Alfred Menezes, Scott Vanstone* 
Guide to Elliptic Curve Cryptography.
*Figure 5.14. Power trace for a sequence of addition (S) and double (D) operations on an elliptic curve over a prime field. Points were represented using Jacobian coordinates. SPA resistance was achieved by insertion of dummy operations in the addition and double algorithms (compare with Figure 5.13). The traces were obtained from an SC140 DSP processor core.*


No es difícil comprobar que son equivalente observando que siempre se cumple que $R_1=a\, R_0$.


A veces es difícil evitar estos tipos de ataques porque los compiladores, al optimizar el código pueden dejar sin efectos las medidas tomadas para evitar los ataques.

[What you get is what you C: Controlling side effects in mainstream C compilers](https://www.cl.cam.ac.uk/~rja14/Papers/whatyouc.pdf), L. Simon, D. Chisnall, R. Anderson:

```
But when a programmer tries to control side effects of code, 
such as to make a cryptographic algorithm execute in constant 
time, the problem remains. 

Programmers devise complex tricks to obscure their intentions, 
but compiler writers find ever smarter ways to optimize code. 

A compiler upgrade can suddenly and without warning open a timing 
channel in previously secure code. This arms race is pointless 
and has to stop.
```







### <a href="#vRSA_2.2.4" id="RSA_2.2.4">Criptograma escogido (reto-respuesta)</a>


Supongamos que un atacante desea descifrar un criptograma  $c\equiv m^e \mod n$ cifrado con la clave pública de $A$.

El atacante elige un entero aleatorio $k$ y calcula $\tilde{c}\equiv c k^e \mod n$. Se pone en contacto con $A$ y le pide que descifre/firme $\tilde{c}$ para que demuestre su identidad. 

$A$ calculará
$$
\tilde{c}^d\equiv  (c k^e)^d \equiv  (m^e)^d k \equiv m\,k \mod n
$$
y enviará $m\,k$ al atacante que sólo tendrá que multiplicar por $k^{-1}\mod n$ para obtener el mensaje $m$.


**Nota:**
	Al usar RSA-OAEP cada $m$ tiene una estructura determinada que si se comprueba que es correcta evita este ataque. 


### <a href="#vRSA_2.2.5" id="RSA_2.2.5">Insuficiente Control de formato</a>

[Bleichenbacher's RSA signature forgery based on implementation error, 2006](https://www.ietf.org/mail-archive/web/openpgp/current/msg00999.html)

[PKCS\#1 Signature Validation](https://www.imperialviolet.org/2014/09/26/pkcs1.html)

Suponganos que para firmar un mensaje el formato de los
datos a firmar es:

`0x00 0x01 0xFF ... 0xFF 0x00 ||  ASN.1 hash del mensaje`

La codificación ASN.1 lleva el tamaño del hash. Algunas implementaciones
no comprueban que tras el hash del mensaje no hay nada más. Esto permite
construir un mensaje a firmar con la siguiente estructura:

`0x00 0x01 0xFF ... 0xFF 0x00 || ASN.1 hash || bytes elegidos oportunamente`

de forma que sea un cubo exacto (fácil de calcular, no es fácil calcular raíces módulo $n$). 

La raíz cúbica pasaría como firma válida de cualquiera que tenga un exponente público igual a 3 y módulo suficientemente grande (el número de bits elegidos oportunamente 
será ligeramente mayor que 2/3, si el exponente es 3, del número de bits del módulo, por lo tanto `0x00 0x01 0xFF ... 0xFF 0x00 ||  ASN.1 hash del mensaje`
ha de tener algo menos de 1/3 del número de bits del módulo) 
ya que al calcular el cubo (verificar la firma) obtendría el hash correcto de la codificación ASN.1 pero no comprobaría que detrás de ella hay más bits.  


In [ ]:
nbits=2048
tipoHash="160160"
hash="00112233445566778899AABBCCDDEEFF00112233"
print ("Tipo de Hash:",tipoHash)
print( "Hash del Mensaje:", hash)
print( "A firmar:"),
m="00"+"01"+(nbits/8-1-1-(len(tipoHash)+len(hash))/2-1)*"ff"+"00"+tipoHash+hash
print( m,"\n")

print( "Falsificando firma:")
c=Integer("0001FFFFFFFFFFFF00"+tipoHash+hash,16)
d=c*2^(nbits-256);
print("000"+hex(d)),"\n"
c1=(floor(d^(1/3))+1)^3
f=c1^(1/3)

print( "Firma:",hex(Integer(f)),"\n")

mensajeRecuperado=hex(Integer(f^3))[2:]
print( "'Hash' del Mensaje recuperado a partir de la Firma:","000"+mensajeRecuperado,"\n")


for i in range(1,len(mensajeRecuperado)):
    if mensajeRecuperado[i]!="f":
        break
tipoHashRecuperado=mensajeRecuperado[i+2:i+2+6]
print(tipoHashRecuperado)

longitudHash=Integer(Integer(mensajeRecuperado[i+2:i+2+3])/4)
hashRecuperado=mensajeRecuperado[i+2+6:i+2+6+longitudHash]

print( "Tipo de Hash recuperado:",tipoHashRecuperado,"\n")
print( "Hash del Mensaje recuperado=",hashRecuperado,"\n")
print( "Firma correcta?", Integer(hash,16)==Integer(hashRecuperado,16),"\n")

### <a href="#vRSA_2.2.6" id="RSA_2.2.6">Error provocado</a>

[D. Boneh, R.A. DeMillo, and R.J. Lipton. *On the Importance of Checking Cryptographic Protocols for Faults*, EUROCRYPT 1997](https://pdfs.semanticscholar.org/7622/200b9459a8c0e25e74ce7316c2402862e919.pdf)

Supongamos un dispositivo que calcula una firma utilizando el teorema chino al que inducimos a que, una vez calculada correctamente la firma módulo $p$, cometa un error al calcular la firma módulo $q$. 

El dispositivo devolverá una firma $S$ correcta módulo $p$ pero incorrecta módulo $q$.
Entonces 
$$
p=\gcd(n,S^e-m)
$$
siendo $m$ el mensaje firmado.



In [ ]:
NdeBits=2048*2; # bits del módulo

e=2^16+1;

r1=randint(0,2^(NdeBits/2));
r2=randint(0,2^(NdeBits/2));
p=next_probable_prime(r1);
while gcd(e,p-1)!=1:
    print( ".");
    p=next_probable_prime(p+2);
q=next_probable_prime(r2);
while gcd(e,q-1)!=1:
    print( "+");
    q=next_probable_prime(q+2);
n=p*q;
phi=(p-1)*(q-1);
d=mod(e^-1,phi);
dp=mod(d,p-1);
dq=mod(d,q-1);
q_inv=mod(q^-1,p);

#print( "p=",p);
#print( "q=",q);
#print "n=",n;
#print "e=",e;
#print "d=",d;
#print "dp=",dp;
#print "dq=",dq;
#print "q_inv=",q_inv;

m=mod(randint(0,2^NdeBits),n);

m_aux=mod(m,p); #Necesario para que trabaje en Zp, si no tarda mucho
fp=mod(m_aux^dp,p);

fq=mod(randint(0,2^NdeBits),q); #Aleatorio, error provocado <---------------------------

h=mod((Integer(fp)-Integer(fq))*(q_inv),n);
f_erronea=mod((Integer(fq)+q*h),n);

f_correcta=mod(m^d,n);


print("\nm=",m)
print("\nfirma erronea=",f_erronea)
#print("firma correcta=",f_correcta)

#-----------Empiezo factorizacion-----------
t0=cputime()
aux=mod(f_erronea^e,n)-m;
primo=gcd(aux,n);
print("\nprimo calculado:",primo)
print("p - primo calculado:", p-primo)
t1=cputime()
print("\n Tiempo=",t1-t0)

### <a href="#vRSA_2.2.7" id="RSA_2.2.7">Primos no aleatorios</a>


* [Ron was wrong, Whit is right. (14 Feb 2012)](https://eprint.iacr.org/2012/064.pdf)
	We performed a sanity check of public keys collected on the web. Our main goal was to test the validity of the assumption that different random choices are made each time keys are generated. We found that the vast majority of public keys work as intended. A more disconcerting finding is that **two out of every one thousand RSA moduli that we collected offer no security.**
    
	Our conclusion is that the validity of the assumption is questionable and that generating keys in the real world for ''multiple-secrets''  cryptosystems such as RSA is significantly riskier than for ''single-secret'' ones such as ElGamal or (EC)DSA which are based on Diffie-Hellman. 
    
    *Factoring one 1024-bit RSA modulus would be historic. Factoring 12720 such moduli is a statistic.*


* [Widespread Weak Keys in Network Devices (2012)](https://factorable.net/)
    We found that 5.57\% of TLS hosts and 9.60\% of SSH hosts share public keys in an apparently vulnerable manner, due to either insufficient randomness during key generation or device default keys.

    We were able to remotely obtain the RSA private keys for 0.50\% of TLS hosts and 0.03\% of SSH hosts because their public keys shared nontrivial common smallfactors due to poor randomness.

    We were able to remotely obtain the DSA private keys for 1.03\% of SSH hosts due to repeated signature randomness.




* [Factoring RSA keys from certified smart cards: Coppersmith in the wild (2013)](https://smartfacts.cr.yp.to/smartfacts-20130916.pdf) 

    This paper explains how an attacker can efficiently factor 184 distinct RSA keys out of more than two million 1024-bit RSA keys downloaded from Taiwan’s national “Citizen Digital Certificate” database. These keys were generated by government-issued smart cards that have built-in hardware random-number generators and that are advertised as having passed FIPS 140-2 Level 2 certification.

    These 184 keys include 103 keys that share primes and that are efficiently factored by a batch-GCD computation. This is the same type of computation that was used last year by two independent teams (USENIX Security 2012: Heninger, Durumeric, Wustrow, Halderman; Crypto 2012: Lenstra, Hughes, Augier, Bos, Kleinjung, Wachter) to factor tens of thousands of cryptographic keys on the Internet.

    The remaining 81 keys do not share primes. Factoring these 81 keys requires taking deeper advantage of randomness-generation failures: first using the shared primes as a springboard to characterize the failures, and then using Coppersmith-type partial-key-recovery attacks. This is the first successful public application of Coppersmith-type attacks to keys found in the wild.



---
#  <a href="#vRSA_3" id="RSA_3">Recomendaciones</a>



* El tamaño de $p$ y $q$ debe de ser de unos 1576 bits y el de $n$ no debe ser inferior a 3072 bits, recomendándose módulos de tamaño mayor.
    
* $\vert p-q\vert \approx p$.

* $p$ y $q$ deben ser primos fuertes. $p$ es primo fuerte si es primo y además:
    - $p-1$ tiene un factor primo grande, $r$;
    - $p+1$ tiene un factor primo grande; 
    - $r-1$ tiene un factor primo grande.

* No hay que compartir módulo.

* El exponente público no debe ser excesivamente pequeño (17 bits como mínimo) y *reconocible*. 
  
* Una clave sólo ha de tener un uso (cifrado, firma, autentificación...)



### Generación de primos fuertes $p$
<p>(1) $p$-1 divisor primo grande $r$ (~1/2 de bits que $p$)</p>
<p>(2) $p$+1 divisor primo grande $s$ (~1/4 de bits que $p$)</p>
<p>(3) $r$-1 divisor primo grande $t$ (~1/4 de bits que $p$)</p>
<p>Primero genero $t$ y $s$.</p>
<p>$r=2\,k\, t+1$, me aseguro la condición (3)</p>
<p>$p_0=2\, s\,s_1-1$,  $s_1$ es el inverso de $s$ módulo $r$</p>
<p>$p=p_0+2\,k\,r\,s$.</p>
<p>Observemos que $p+1=2\,s\, s_1+2\,k\,r\,s$.</p>
<p>Notemos que $p-1=2\,s\,s_1-2+2\,k\,r\, s = 2(s\,s_1-1+k\,r\,s)$, si tomamos módulo $r$, teniendo en cuenta que $s_1$ es el inverso de $s$ módulo $r$ nos queda que $p-1\equiv 0\mod r$. Por lo tanto $p-1$ es un múltiplo de $r$.</p>



<p><em>p 1024 bits ~ 5 segundos</em></p>
<p><em>p 2048 bits ~ 50-90 segundos</em></p>
<p><em><br /></em></p>

In [ ]:
NdeBitsDelPrimoFuerte=1024*1;
NdeBitsDelPrimo=NdeBitsDelPrimoFuerte/4;
aux=randint(0,2^NdeBitsDelPrimo);
t=next_probable_prime(aux);
print("t=",t, flush=True)

aux=randint(0,2^NdeBitsDelPrimo);
s=next_probable_prime(aux);
print("s=",s, flush=True)

aux=randint(0,2^(NdeBitsDelPrimo));
r=aux*2*t+1;
while not(is_prime(r)):
    aux=aux+1;
    r=aux*2*t+1;
print("r=",r, flush=True)

t0 = cputime()

p0= 2*s*(s.powermod(r-2,r)) -1;
aux=randint(0,2^(NdeBitsDelPrimo));
p=p0+aux*2*r*s;
while not(is_prime(p)):
    aux=aux+1
    p=p0+aux*2*r*s;
print("\np=",p, flush=True)

t1=cputime()
print("\n Tiempo=",t1-t0)

print(mod(p-1,r), is_prime(r));
print(mod(p+1,s), is_prime(s));
print(mod(r-1,t), is_prime(t));
print(is_prime(p));